# Colección de datos para modo libre - **Anthony**
---
La idea es que Anthony pueda circular libremente por un espacio X, donde este esté atento a objetos en el camino y poderlos evitar. La idea es que el robot gire en su propio eje hasta que encuentre un lugar seguro para caminar, esto ayuda a que no tenga un choque al dar la vuelta del primer objeto localizado.

Por lo tanto, se requiere capturar información que le permita a Anthony tener conocimiendo de donde puede avanzar y donde no.
Se tomarán fotos desde la cámara del robot para ayudarlo con su entrenamiento, primero captando lugares donde este no puede transitar, seguidamente de donde puede.





# Transmisión en vivo desde la cámara de Anthony
---
Se inicia la cámara y se visualiza en el notebook. Se requiere por temas de costo de recursos, minimizar el tamaño de las imágenes o moldear la cámara para que tome las fotos en un tamaño de 224x224.

In [ ]:
# imports necesarios
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224) # se instancia la cámara al tamaño establecido (224x224)

image = widgets.Image(format='jpeg', width=224, height=224)  # formato jpeg de 224x224

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)


Se desplega la cámara (en caso de querer ver su funcionamiento, nuevamente se mostrará más adelante)

In [ ]:
display(image)

# Creación de las carpetas que conteneran las imagenes

In [ ]:
import os

blocked_dir = 'dataset/blocked' # set de datos de imágenes con bloqueos para el robot
free_dir = 'dataset/free' # set de datos de imágenes con panorama libre

try: # intenta crear las carpetas, ya que si existen esto provocaría un error
    os.makedirs(free_dir)
    os.makedirs(blocked_dir)
except FileExistsError: # en caso de fallo las carpetas ya existían
    print('Las carpetas no fueron creadas, estas ya existen.')

# Creación de herramientas para la toma de imágenes
---
Una vez creados los directorios, se crearán los botones que facilitarán la obtención de las imágenes (uno para las de bloqueo y otro para las libre). 
Además, se crearán dos contadores con la cantidad de imágenes que se han tomado hasta el momento.

In [ ]:
button_layout = widgets.Layout(width='100px', height='64px')

# se crean los botones que permitirán la toma de fotos
free_button = widgets.Button(description='Libre', button_style='success', layout=button_layout)
blocked_button = widgets.Button(description='Bloqueo', button_style='danger', layout=button_layout)

# se crean los contadores de archivos
free_count = widgets.IntText(layout=button_layout, value=len(os.listdir(free_dir)))
blocked_count = widgets.IntText(layout=button_layout, value=len(os.listdir(blocked_dir)))

# se muestran los botones para agregar las imágenes con su respectivo contador
display(widgets.HBox([free_button]))
display(widgets.HBox([free_count]))
display(widgets.HBox([blocked_button]))
display(widgets.HBox([blocked_count]))

Creación de las funciones que permitirán el guardado de la información, así como los agregar los listener a los botones

Para asegurarnos de no repetir ningún nombre de archivo. Se usara el paquete 'uuid' en python, que define el método  para generar un identificador único. Este identificador único se genera a partir de información como la hora actual y la dirección de la máquina.

In [ ]:
from uuid import uuid1

def save_snapshot(directory):
    image_path = os.path.join(directory, str(uuid1()) + '.jpg')
    with open(image_path, 'wb') as f:
        f.write(image.value)

def save_free():
    global free_dir, free_count
    save_snapshot(free_dir)
    free_count.value = len(os.listdir(free_dir))
    
def save_blocked():
    global blocked_dir, blocked_count
    save_snapshot(blocked_dir)
    blocked_count.value = len(os.listdir(blocked_dir))
    
# se agregan los listeners
free_button.on_click(lambda x: save_free())
blocked_button.on_click(lambda x: save_blocked())

A continuación se visualizarán las herramientas previamente creadas, en conjunto con la imagen que Anthony está capturando.

In [ ]:
display(image)

display(widgets.HBox([free_button]))
display(widgets.HBox([free_count]))

display(widgets.HBox([blocked_button]))
display(widgets.HBox([blocked_count]))

Si se quiere detener el uso de la cámara

In [ ]:
camera.stop()

# Comprimir dataset de imágenes en .zip


In [ ]:
!zip -r -q dataset.zip dataset